# Imports

In [ ]:
# loading libraries for data manipulation
import numpy as np
import pandas as pd

# loading libraries for data visualization
import matplotlib.pyplot as plt
from plotnine import *
from PIL import Image

# misc stuff
from sklearn.model_selection import train_test_split

# import tensorflow and keras packages
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Flatten, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception

import warnings
warnings.filterwarnings('ignore')

import os,glob,shutil

# Data Importing

In [2]:
cancer_train_source = "../datasets/Skin_Data/Cancer/Training"
cancer_test_source = "../datasets/Skin_Data/Cancer/Testing"
non_cancer_train_source = "../datasets/Skin_Data/Non_Cancer/Training"
non_cancer_test_source = "../datasets/Skin_Data/Non_Cancer/Testing"

# **Hedef klasörler (Kaggle Working dizininde yeni dataset için)**
cancer_train_target = "../datasets/Skin_Data/Training/Cancer"
cancer_test_target = "../datasets/Skin_Data/Testing/Cancer"
non_cancer_train_target = "../datasets/Skin_Data/Training/Non_Cancer"
non_cancer_test_target = "../datasets/Skin_Data/Testing/Non_Cancer"

# **Hedef klasörleri oluştur**
os.makedirs(cancer_train_target, exist_ok=True)
os.makedirs(cancer_test_target, exist_ok=True)
os.makedirs(non_cancer_train_target, exist_ok=True)
os.makedirs(non_cancer_test_target, exist_ok=True)

# **Verileri kopyala**
def copy_images(source_folder, target_folder):
    if os.path.exists(source_folder):
        for file_name in os.listdir(source_folder):
            src_path = os.path.join(source_folder, file_name)
            dst_path = os.path.join(target_folder, file_name)
            if src_path.lower().endswith(('png', 'jpg', 'jpeg')):  # Sadece resim dosyalarını kopyala
                shutil.copy(src_path, dst_path)

# **Orijinal resimleri Kaggle Input'tan Kaggle Working'e taşıyoruz**
copy_images(cancer_train_source, cancer_train_target)
copy_images(cancer_test_source, cancer_test_target)
copy_images(non_cancer_train_source, non_cancer_train_target)
copy_images(non_cancer_test_source, non_cancer_test_target)

In [5]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True
)
# have to augment data bcs there aren't many images in the directory 
def augment_images(source_folder, num_images):
    generator = datagen.flow_from_directory(
        os.path.dirname(source_folder),  
        target_size=(224, 224),
        batch_size=1,
        class_mode=None,
        save_to_dir=source_folder,  
        save_prefix="aug",
        save_format='jpeg'
    )

    for i in range(num_images):
        next(generator)


augment_images(cancer_train_target, 200)  
augment_images(cancer_test_target, 100)  


augment_images(non_cancer_train_target, 200) 
augment_images(non_cancer_test_target, 100)  

Found 84 images belonging to 2 classes.
Found 204 images belonging to 2 classes.
Found 284 images belonging to 2 classes.
Found 304 images belonging to 2 classes.


In [6]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = train_datagen.flow_from_directory(
    "../datasets/Skin_Data/Training",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    "../datasets/Skin_Data/Training",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 388 images belonging to 2 classes.
Found 96 images belonging to 2 classes.


# Network Training

In [7]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.Conv2D(128, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(128, (3,3), activation='relu'),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=1e-3)
# compile model
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 107, 107, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 53, 53, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 51, 51, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 49, 49, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,359,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,674,049 (10.20 MB)

 Trainable params: 2,674,049 (10.20 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# fit model
history = model.fit(
    train_data,
    epochs=10,
    batch_size=64,
    validation_data=val_data
)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.5315 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.5230 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.4483 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.5086 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.4791 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.4956 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.5072 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.4932 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss:

# Transfer Learning

In [43]:
# Load the pre-trained resnet model without the top classification layer
xception_base = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
xception_base.trainable = False  

model2 = keras.models.Sequential([
    xception_base,
    keras.layers.AveragePooling2D(pool_size=(7, 7)),
    keras.layers.Flatten(),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=1e-3)
model2.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model2.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_4             │ (None, 1, 1, 2048)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 200)            │       409,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,271,481 (81.14 MB)

 Trainable params: 410,001 (1.56 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [32]:
early_stop = EarlyStopping(
    monitor='val_loss',  
    patience=5,          
    restore_best_weights=True  
)

history2 = model2.fit(
    train_data,  
    validation_data=val_data,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 17s 551ms/step - accuracy: 0.5127 - loss: 0.9621 - val_accuracy: 0.5341 - val_loss: 1.0434
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 364ms/step - accuracy: 0.6031 - loss: 0.7523 - val_accuracy: 0.5625 - val_loss: 0.8005
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 377ms/step - accuracy: 0.6511 - loss: 0.6514 - val_accuracy: 0.5341 - val_loss: 0.8446
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 374ms/step - accuracy: 0.6427 - loss: 0.6560 - val_accuracy: 0.5398 - val_loss: 0.8191
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 373ms/step - accuracy: 0.6271 - loss: 0.6777 - val_accuracy: 0.5170 - val_loss: 1.0320
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 382ms/step - accuracy: 0.6370 - loss: 0.6377 - val_accuracy: 0.5511 - val_loss: 0.7889
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 374ms/step - accuracy: 0.6582 - loss: 0.6106 - val_accuracy: 0.5341 - val_loss: 0.8290
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 378ms/step - accuracy: 0.6949 - loss: 0.5839 - val_accuracy: 0